# Exercise Set 1

## Multiple Choice / True-False

(Ehsan and Dan) Some simple questions based on the prior sessions
....

## Simple modeling / coding

1. Show these constraints are either compatible (find a feasible solution) or incompatible (prove a contradiction) algebraically

\begin{align*}
x + y + z &\ge 3    \\
2x + 3y - z &\le 4  \\
x, y, z &\ge 0
\end{align*}

\begin{align*}
x_1 + x_2 + x_3 &= 6       \\
3x_1 + 2x_2 + x_3 &\ge 20  \\
x_1, x_2, x_3, &\ge 0
\end{align*}

2. Translate the following into constraints using binary variables for deciding to invest in three stocks. Then determine if they are compatible as above. 

- I will invest in exactly two stocks. 
- If I invest in stock A, then I will invest in stock B
- If I invest in stock B, then I will invest in stock C


## Back to the widget production problem
Use the code below for as the base model for this problem. 

In [2]:
# Import packages
import pandas as pd
import gurobipy as gp
from gurobipy import GRB

In [6]:
# read in transportation cost data
path = 'https://raw.githubusercontent.com/Gurobi/modeling-examples/master/optimization101/Modeling_Session_1/'
transp_cost = pd.read_csv(path + 'cost.csv')

# get production and distribution locations from data frame
production = list(transp_cost['production'].unique())
distribution = list(transp_cost['distribution'].unique())
transp_cost = transp_cost.set_index(['production','distribution']).squeeze()

max_prod = pd.Series([180,200,140,80,180], index = production, name = "max_production")
n_demand = pd.Series([89,95,121,101,116,181], index = distribution, name = "demand")
# the min production is a fraction of the max
frac = 0.75
C = 30


In [7]:
# gurobipy code for model
m = gp.Model('widgets')
m.setParam('OutputFlag',0)

# decision vars
x = m.addVars(production, distribution, vtype=GRB.SEMICONT, lb = C, name = 'prod_ship')

# constraints
can_produce = m.addConstrs((gp.quicksum(x[p,d] for d in distribution) <= max_prod[p] for p in production), name = 'can_produce')
must_produce = m.addConstrs((gp.quicksum(x[p,d] for d in distribution) >= frac*max_prod[p] for p in production), name = 'must_produce')
meet_demand = m.addConstrs(x.sum('*', d) >= n_demand[d] for d in distribution)

#objective
m.setObjective(gp.quicksum(transp_cost[p,d]*x[p,d] for p in production for d in distribution), GRB.MINIMIZE)

m.optimize()

x_values = pd.Series(m.getAttr('X', x), name = "shipment", index = transp_cost.index)
sol = pd.concat([transp_cost, x_values], axis=1)

print(f"The original model had a total cost of {round(m.ObjVal,2)}")
sol[sol.shipment > 0]

The original model had a total cost of 1748.42


cost  shipment
production  distribution                
Baltimore   Nashville     5.96      30.0
            Richmond      1.96     116.0
Cleveland   Columbia      2.43      89.0
            Indianapolis  2.37      95.0
Little Rock St. Louis     2.92     140.0
Birmingham  Nashville     1.53      71.0
Charleston  Lexington     1.61     121.0
            St. Louis     4.60      41.0

## Nonlinear optimization modeling

### Quadratic expression

In order to produce more widgets, production facilities must run their machines at faster speeds. This consumes more energy and produces more waste at a level that grows with each widget produced. Running their machines costs a production facility $0.01 times the square of the number of widgets produced. Modify the objective function to include this additional production cost. Here is the some code to get you started - it includes the original transportation cost expression that you will be adding to:

```python
m.setObjective(gp.quicksum(transp_cost[p,d]*x[p,d] for p in production for d in distribution) + ..., GRB.MINIMIZE)
```

In [8]:
# One solution
m.setObjective(gp.quicksum(transp_cost[p,d]*x[p,d] for p in production for d in distribution) + 0.01 * gp.quicksum(x.sum(p, '*') * x.sum(p, '*') for p in production), GRB.MINIMIZE)

# Another solution
# Introduces auxiliary variables representing total production for each facility
# produced = m.addVars(production, name='total_produced')
# m.addConstrs((produced[p] == x.sum(p, '*') for p in production), name='set_produced')
# m.setObjective(gp.quicksum(transp_cost[p,d]*x[p,d] for p in production for d in distribution) + 0.01 * gp.quicksum(produced[p] * produced[p] for p in production), GRB.MINIMIZE)

### Nonlinear expression 1

The widgets will now be shipped using hyper-efficient vehicles. The cost to ship $x_{ij}$ widgets from production facility $i$ to distribution center $j$ is now equal to the corresponding transportation cost times $\log(1 + x_{ij})$. Modify the objective function to be equal to this new transportation cost. Some code to get you started:

```python
prod_cost = m.addVar(name='prod_cost')
m.addConstr(prod_cost ==..., name='set_prod_cost')
m.setObjective(prod_cost, GRB.MINIMIZE)
```

In [ ]:
# One solution
prod_cost = m.addVar(name='prod_cost')
m.addConstr(prod_cost == gp.quicksum(transp_cost[p,d] * gp.nlfunc.log(1 + x[p, d]) for p in production for d in distribution), name='set_prod_cost')
m.setObjective(prod_cost, GRB.MINIMIZE)

### Nonlinear expression 2

A very strange company rule stipulates that the following mathematical relationship must hold:

$ x_{\textrm{Cleveland,Richmond}} = \log\Large(\frac{10 \cdot x_{\textrm{Baltimore,Indianapolis}}}{\sum_d x_{\textrm{Baltimore},d}})\normalsize + x_{\textrm{Cleveland,Richmond}}$

Add this exact constraint to the model using a nonlinear expression.

In [ ]:
# One solution
m.addConstr(x['Baltimore', 'Richmond'] == gp.nlfunc.log(10 * x['Baltimore', 'Richmond'] / x.sum('Baltimore', '*')) + x['Cleveland', 'Richmond'], name='strange_company_rule')

 ### Linearizing a nonlinear expression

Wait a minute...is the nonlinear constraint we just added unnecessarily complicated? Rewrite the nonlinear constraint using only linear expressions. Describe the purpose of the constraint in words.

In [ ]:
# One solution
# Purpose: exactly 10% of Baltimore's production must be sent to Indianapolis
m.addConstr(x['Baltimore', 'Indianapolis'] == 0.1 * x.sum('Baltimore', '*'), name='strange_company_rule_linearized')